In [98]:
%load_ext tensorboard
import os
os.environ["TF_CPP_MIN_VLOG_LEVEL"] = "2"
import tensorflow as tf
import random
import numpy as np
import pandas as pd
import tensorrt as trt

print("TF:", tf.__version__)
print("GPUs:", tf.config.list_physical_devices('GPU'))
print("TensorRt:", trt.__version__)

RANDOM_STATE = 42

random.seed(RANDOM_STATE)
tf.random.set_seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)

TARGET = "rucwar"
MIN_PER_CLASS = 10
OUTER_SPLITS = 5
INNER_SPLITS = 3
BATCH_SIZE = 32

SAMPLE_RATE = 16000
MAX_DURATION = 10 # seconds
NEG_POS_RATIO = 3
TRAIN_NEG_RATIO = 2
VAL_NEG_RATIO = 2
FILL_TYPE = "pad" # pad | tile
N_FRAMES = 64
FRAME_OVERLAP = int(np.ceil(N_FRAMES * 0.5))

SPECTROGRAM_SECONDS = 3

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
TF: 2.20.0
GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
TensorRt: 10.13.2.6


In [99]:
from sklearn.model_selection import StratifiedGroupKFold
import os
import librosa

# birdclef sanity checks
def sanity_birdlcef(df: pd.DataFrame, target: str):
    # File integrity
    for r in df.itertuples():
        if os.path.isfile(r.path) is not True:
            print(f"[ERROR]: {r.path} is not valid!")
    
    # Dupicates
    dups = df['path'].duplicated().sum()  # or 'filename'
    print("Duplicate file rows:", dups)
    
    # Count summary
    print(f"Total Positives: {df[df['primary_label'] == target].shape[0]}")
    print(f"Total Negatives: {df[df['primary_label'] != target].shape[0]}")
    print(f"Avg samples per Class: {df['primary_label'].value_counts().mean():.0f}")



# Different functions for each dataset
def load_birdclef(audio_root, path, target, min_per_class = MIN_PER_CLASS):
    df = pd.read_csv(path)
    
    df["path"] = audio_root + "/" + df["primary_label"] + "/" + df["filename"]
    
    # Optional: drop rare classes (keeps CV stable)
    if min_per_class > 1:
        keep_labels = df["primary_label"].value_counts()
        keep_labels = keep_labels[keep_labels >= min_per_class].index
        df = df[df["primary_label"].isin(keep_labels)].reset_index(drop=True)
    
    # Perform sanity checks
    sanity_birdlcef(df, target)

    # Generate the binary labels, target = 1 else 0
    labels = (df['primary_label'] == target).astype(int).values
    
    # Group based on auther + time te prevent straddeling
    df["group_key"] = df["author"] + df["time"]
    
    return df
    

os.chdir("/home/joris/Thesis/new_attempt")

birdclef_df = load_birdclef("datasets/birdclef_2021/train_short_audio", "datasets/birdclef_2021/train_metadata.csv", target=TARGET)


Duplicate file rows: 0
Total Positives: 154
Total Negatives: 62695
Avg samples per Class: 160


In [100]:
from sklearn.model_selection import StratifiedGroupKFold
import numpy as np
import pandas as pd

def make_nested_cv_splits(
    df: pd.DataFrame,
    target: str = TARGET,
    outer_splits: int = OUTER_SPLITS,
    inner_splits: int = INNER_SPLITS,
    random_state: int = RANDOM_STATE,
):
    """
    Build nested CV folds for binary BirdCLEF: target species = 1, others = 0.
    Uses StratifiedGroupKFold for both outer and inner splits to avoid leakage across groups.
    """
    
    # Binary labels for stratification
    y_all = (df["primary_label"] == target).astype(int).values
    groups_all = df["group_key"].astype(str).fillna("NA").values
    idx_all = np.arange(len(df))

    outer_kf = StratifiedGroupKFold(n_splits=outer_splits, shuffle=True, random_state=random_state)

    nested = []
    for k, (outer_tr_idx, outer_te_idx) in enumerate(outer_kf.split(idx_all, y=y_all, groups=groups_all), start=1):
        # Outer train/val pool and test set
        trval_idx = idx_all[outer_tr_idx]
        test_idx  = idx_all[outer_te_idx]

        y_trval   = y_all[outer_tr_idx]
        groups_trval = groups_all[outer_tr_idx]

        # Inner CV on the outer train/val pool
        inner_kf = StratifiedGroupKFold(n_splits=inner_splits, shuffle=True, random_state=random_state)
        inner_folds = []
        for j, (inner_tr_rel, inner_va_rel) in enumerate(inner_kf.split(trval_idx, y=y_trval, groups=groups_trval), start=1):
            # Map relative indices back to global indices
            inner_tr_idx = trval_idx[inner_tr_rel]
            inner_va_idx = trval_idx[inner_va_rel]

            inner_folds.append({
                "inner_fold": j,
                "inner_train_idx": inner_tr_idx,
                "inner_val_idx":   inner_va_idx,
            })

        nested.append({
            "outer_fold": k,
            "outer_train_idx": trval_idx,
            "outer_test_idx":  test_idx,
            "inner_folds": inner_folds,
            "train_pos_ratio": float(y_all[outer_tr_idx].mean()),
            "test_pos_ratio":  float(y_all[outer_te_idx].mean()),
        })

    return nested

    
    
cross_validation_sets = make_nested_cv_splits(birdclef_df)

In [101]:
import tensorflow_io as tfio
import subprocess

def load_ogg_ffmpeg(path, sr=16000):
    path = path.decode("utf-8")
    cmd = [
        "ffmpeg", "-i", path, "-f", "f32le",
        "-ac", "1", "-ar", str(sr), "pipe:1", "-loglevel", "quiet"
    ]
    out = subprocess.check_output(cmd)
    audio = np.frombuffer(out, np.float32)
    return audio

def load_ogg_librosa(path, sr=16000):
    path = path.decode("utf-8")
    y, sr = librosa.load(path, sr=SAMPLE_RATE)
    return y


def audio_pipeline(filename, augment=False, gaussian=0):
    # Load audio file as tensor 
    # audio_file = tf.numpy_function(load_ogg_ffmpeg, [filename, SAMPLE_RATE], tf.float32)
    audio_file = tf.numpy_function(load_ogg_librosa, [filename, SAMPLE_RATE], tf.float32)
    # audio_file, sr = librosa.load(filename.numpy(), sr=SAMPLE_RATE)
    
    # Remove last dimension
    waveform = audio_file[:SAMPLE_RATE * MAX_DURATION]
    
    if False:
        # Trim the noise from the audio
        position = tfio.audio.trim(waveform, axis=0, epsilon=0.1, name="Trim")
        
        start = position[0]
        stop = position[1]

        processed = waveform[start:stop]
    else:
        processed = waveform[:SAMPLE_RATE * SPECTROGRAM_SECONDS]
        
    # if gaussian > 0:
    #     processed = aug_gaussian_noise_snr(processed, gaussian)
    
    frame_count = tf.shape(processed)
    # Calculate end padding
    if (frame_count < SAMPLE_RATE):
        processed = waveform
        frame_count = tf.shape(processed)
    
    if FILL_TYPE == "pad":
        padding = tf.maximum(0, (SAMPLE_RATE * SPECTROGRAM_SECONDS) - frame_count[0])
        processed = tf.pad(processed, paddings=[[padding, 0]], name="Padding")
    elif FILL_TYPE == "tile":
        repeats = tf.maximum(tf.cast(1, tf.int64), 1 + tf.cast(((SAMPLE_RATE * SPECTROGRAM_SECONDS) / frame_count), tf.int64))
        
        processed = tf.repeat(processed, repeats)
        # Reduce size to max 
        processed = processed[:SAMPLE_RATE * SPECTROGRAM_SECONDS]
        
    if augment:
        # Fade in and out
        fade = tfio.audio.fade(
            processed, fade_in=1000, fade_out=2000, mode="logarithmic", name="Fade")
    else:
        fade = processed
    
    # Band filter
    from scipy import signal
    b, a = signal.butter(4, [200, 7999], fs=SAMPLE_RATE, btype='band')
    band_filter = tf.py_function(signal.lfilter, [b, a, fade], Tout=tf.float32, name="Filter")

    spectrogram = tfio.audio.spectrogram(band_filter, nfft=1024, window=512, stride=256)
    mel_spectrogram = tfio.audio.melscale(spectrogram, rate=SAMPLE_RATE, mels=128, fmin=20, fmax=8000)
    db_mel_spectrogram = tfio.audio.dbscale(mel_spectrogram, top_db=80)
    
    db_mel_spectrogram = tf.expand_dims(db_mel_spectrogram, -1)
    
    db_mel_spectrogram = tf.transpose(db_mel_spectrogram, perm=[1, 0 ,2])
    
    # db_mel_spectrogram = tf.ensure_shape(db_mel_spectrogram, (128, 188, 1))
    
    return db_mel_spectrogram

In [102]:
import tensorflow as tf

# --------- Pick a Binary Focal Cross-Entropy loss safely ----------
def get_binary_focal_loss(gamma=2.0, alpha=0.25):
    # 1) Native Keras (TF ≥ 2.12-ish)
    if hasattr(tf.keras.losses, "BinaryFocalCrossentropy"):
        return tf.keras.losses.BinaryFocalCrossentropy(gamma=gamma, alpha=alpha, from_logits=False)
    # 2) TensorFlow Addons fallback
    try:
        import tensorflow_addons as tfa
        return tfa.losses.SigmoidFocalCrossEntropy(gamma=gamma, alpha=alpha)
    except Exception:
        # 3) Minimal custom fallback (y_true∈{0,1}, y_pred∈[0,1])
        def focal_bce(y_true, y_pred):
            eps = tf.keras.backend.epsilon()
            y_pred = tf.clip_by_value(y_pred, eps, 1. - eps)
            # standard BCE parts
            ce_pos = -tf.math.log(y_pred)
            ce_neg = -tf.math.log(1. - y_pred)
            # focal weighting
            loss_pos = alpha * tf.pow(1. - y_pred, gamma) * ce_pos * y_true
            loss_neg = (1. - alpha) * tf.pow(y_pred, gamma) * ce_neg * (1. - y_true)
            return tf.reduce_mean(loss_pos + loss_neg)
        return focal_bce

# --------- Simple binary CNN ----------
def build_binary_cnn(
    input_shape=(128, 64, 1),
    lr=1e-3,
    l2=1e-4,
    dropout=0.25,
    gamma=2.0,
    alpha=0.25,
):
    """
    Binary classifier for log-mel spectrograms (target vs non-target).
    Output: single sigmoid unit.
    Loss: Binary Focal Cross-Entropy (with safe fallbacks).
    """
    L2 = tf.keras.regularizers.l2(l2)
    Conv = tf.keras.layers.Conv2D
    focal_loss = get_binary_focal_loss(gamma=gamma, alpha=alpha)

    inputs = tf.keras.Input(shape=input_shape)

    # Block 1
    x = Conv(32, (3, 3), padding="same", kernel_regularizer=L2)(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = Conv(32, (3, 3), padding="same", kernel_regularizer=L2)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)   # 128x64 -> 64x32
    x = tf.keras.layers.Dropout(dropout)(x)

    # Block 2
    x = Conv(64, (3, 3), padding="same", kernel_regularizer=L2)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = Conv(64, (3, 3), padding="same", kernel_regularizer=L2)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)   # 64x32 -> 32x16
    x = tf.keras.layers.Dropout(dropout)(x)

    # Block 3
    x = Conv(96, (3, 3), padding="same", kernel_regularizer=L2)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = Conv(96, (3, 3), padding="same", kernel_regularizer=L2)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)   # 32x16 -> 16x8
    x = tf.keras.layers.Dropout(dropout)(x)

    # Head
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(128, activation="relu", kernel_regularizer=L2)(x)
    x = tf.keras.layers.Dropout(dropout)(x)
    outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)  # binary

    model = tf.keras.Model(inputs, outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
        loss=focal_loss,
        metrics=[
            tf.keras.metrics.BinaryAccuracy(name="acc", threshold=0.5),
            tf.keras.metrics.AUC(curve="ROC", name="auc"),
            tf.keras.metrics.AUC(curve="PR",  name="auprc"),
            tf.keras.metrics.Precision(name="precision", thresholds=0.5),
            tf.keras.metrics.Recall(name="recall", thresholds=0.5),
        ],
    )
    return model


In [103]:
import random
from typing import Sequence, Optional, Tuple

def _map_pos(x, y):
    return audio_pipeline(x, augment=True), y

def _map_neg(x, y):
    return audio_pipeline(x, augment=True), y

def plan_epoch_counts(n_pos_train: int, neg_pos_ratio: float = 2.0) -> Tuple[int, int]:
    P_pos = int(n_pos_train)               # see each positive once per epoch
    P_neg = int(round(neg_pos_ratio * P_pos))
    return P_pos, P_neg 

def sample_train_negatives(neg_all: Sequence[str], n_neg: int, seed: Optional[int] = None) -> Sequence[str]:
    n = min(n_neg, len(neg_all))
    rng = random.Random(seed)
    return rng.sample(list(neg_all), n) if n > 0 else []

def make_fixed_val_negatives(neg_all: Sequence[str], n_pos_val: int, neg_pos_ratio: int = NEG_POS_RATIO, seed: int = RANDOM_STATE) -> Sequence[str]:
    n_neg = min(neg_pos_ratio * n_pos_val, len(neg_all))
    rng = random.Random(seed)
    return rng.sample(list(neg_all), n_neg) if n_neg > 0 else []

def build_train_dataset(pos_files: Sequence[str], neg_files: Sequence[str], batch_size: int = BATCH_SIZE, shuffle: bool = True) -> tf.data.Dataset:
    labels_pos = tf.ones([len(pos_files)], dtype=tf.float32)
    labels_neg = tf.zeros([len(neg_files)], dtype=tf.float32)
    
    ds_pos = tf.data.Dataset.from_tensor_slices((list(pos_files), labels_pos)).map(_map_pos, num_parallel_calls=10)
    ds_neg = tf.data.Dataset.from_tensor_slices((list(neg_files), labels_neg)).map(_map_neg, num_parallel_calls=10)

    ds = ds_pos.concatenate(ds_neg)
    if shuffle:
        ds = ds.shuffle(buffer_size=len(pos_files) + len(neg_files), seed=RANDOM_STATE, reshuffle_each_iteration=True)
    ds = ds.batch(batch_size).cache()
    return ds


def build_val_dataset(pos_files: Sequence[str], neg_files_fixed: Sequence[str], batch_size: int = BATCH_SIZE) -> tf.data.Dataset:
    y_pos = tf.ones([len(pos_files)],        dtype=tf.float32)
    y_neg = tf.zeros([len(neg_files_fixed)], dtype=tf.float32)

    ds_pos = tf.data.Dataset.from_tensor_slices((list(pos_files), y_pos)).map(lambda x,y: (audio_pipeline(x, augment=False), y), num_parallel_calls=10)
    ds_neg = tf.data.Dataset.from_tensor_slices((list(neg_files_fixed), y_neg)).map(lambda x,y: (audio_pipeline(x, augment=False), y), num_parallel_calls=10)

    return ds_pos.concatenate(ds_neg).batch(batch_size).cache()

def build_file_lists(df: pd.DataFrame, idx, target=TARGET):
    sub = df.iloc[idx]
    pos = sub[sub.primary_label == target]["path"].tolist()
    neg = sub[sub.primary_label != target]["path"].tolist()
    return pos, neg

def make_epoch_train_dataset(pos_tr_all: Sequence[str], neg_tr_all: Sequence[str],
                             neg_pos_ratio: float = 2.0, batch_size: int = BATCH_SIZE, seed: Optional[int] = RANDOM_STATE) -> tf.data.Dataset:
    P_pos, P_neg = plan_epoch_counts(len(pos_tr_all), neg_pos_ratio)
    neg_epoch = sample_train_negatives(neg_tr_all, P_neg, seed=seed)
    return build_train_dataset(pos_tr_all, neg_epoch, batch_size=batch_size, shuffle=True)

def make_fixed_val_dataset(pos_va_all: Sequence[str], neg_va_all: Sequence[str],
                           neg_pos_ratio: int = 3, batch_size: int = BATCH_SIZE, seed: int = RANDOM_STATE) -> tf.data.Dataset:
    neg_fixed = make_fixed_val_negatives(neg_va_all, len(pos_va_all), neg_pos_ratio=neg_pos_ratio, seed=seed)
    return build_val_dataset(pos_va_all, neg_fixed, batch_size=batch_size)


def build_all_file_lists(df: pd.DataFrame, folds: dict, epoch: int = 0):
    out = []
    for fold in folds:
        outer_fold = int(fold["outer_fold"])
        outer_train_idx = fold["outer_train_idx"]
        outer_test_idx = fold["outer_test_idx"]
    
        # Build the file lists for the outer fold
        pos_tr_all, neg_tr_all = build_file_lists(df, outer_train_idx)    
        pos_test_all, neg_test_all = build_file_lists(df, outer_test_idx)
    

        # Rotate negatives each epoch via seed that depends on (outer, inner, epoch)
        outer_train_ds = make_epoch_train_dataset(
            pos_tr_all,
            neg_tr_all,
            neg_pos_ratio=TRAIN_NEG_RATIO,
            batch_size=BATCH_SIZE,
            seed=outer_fold,
        )

        outer_test_ds = make_fixed_val_dataset(
            pos_test_all,
            neg_test_all,
            neg_pos_ratio=VAL_NEG_RATIO,
            batch_size=BATCH_SIZE,
            seed=outer_fold,  # fixed per outer fold
        )
    
        
        inner_list = []
        for inner in fold["inner_folds"]:
            inner_fold = int(inner["inner_fold"])
            inner_train_idx = inner["inner_train_idx"]
            inner_val_idx   = inner["inner_val_idx"]

            pos_tr_all, neg_tr_all = build_file_lists(df, inner_train_idx)
            pos_va_all, neg_va_all = build_file_lists(df, inner_val_idx)

            # Rotate negatives each epoch via seed that depends on (outer, inner, epoch)
            train_ds = make_epoch_train_dataset(
                pos_tr_all,
                neg_tr_all,
                neg_pos_ratio=TRAIN_NEG_RATIO,
                batch_size=BATCH_SIZE,
                seed=outer_fold * 100_000 + inner_fold * 1_000 + epoch,
            )

            # Fixed validation and test datasets (reproducible seeds per outer/inner)
            val_ds = make_fixed_val_dataset(
                pos_va_all,
                neg_va_all,
                neg_pos_ratio=VAL_NEG_RATIO,
                batch_size=BATCH_SIZE,
                seed=outer_fold * 100_000 + inner_fold,
            )

            test_ds = make_fixed_val_dataset(
                pos_test_all,
                neg_test_all,
                neg_pos_ratio=VAL_NEG_RATIO,
                batch_size=BATCH_SIZE,
                seed=outer_fold,  # fixed per outer fold
            )

            inner_list.append({
                "inner_fold": inner_fold,
                "train_ds": train_ds,
                "val_ds":   val_ds,
                "test_ds":  test_ds,
            })
        out.append({"outer_fold": outer_fold, "train_ds":outer_train_ds, "test_ds":outer_test_ds, "inner": inner_list})
    return out
        

datasets = build_all_file_lists(birdclef_df, cross_validation_sets)

In [104]:
%reload_ext tensorboard
%tensorboard --bind_all --logdir

per_fold_results = []

EPOCHS_PER_FOLD = 5

hparams_per_fold = {
    0: {"lr":1e-3, "gamma":1.0, "alpha":0.25},
    1: {"lr":1e-3, "gamma":2.0, "alpha":0.5},
    2: {"lr":1e-3, "gamma":3.0, "alpha":0.75},
    3: {"lr":1e-3, "gamma":4.0, "alpha":0.25},
    4: {"lr":1e-3, "gamma":5.0, "alpha":0.25},
}

tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = "logs/",
                                                 histogram_freq = 1,
                                                 profile_batch = '1, 4')

for outer_fold in datasets:
    print(f"[INFO]: Running outer fold: {outer_fold["outer_fold"]}")
    inner_scores = []
    for inner_fold in outer_fold["inner"]:
        print(f"[INFO]: Running inner fold: {inner_fold["inner_fold"]}")
        model = build_binary_cnn(input_shape=(128, 188, 1), 
                                 lr=hparams_per_fold[inner_fold["inner_fold"]]["lr"], 
                                 gamma=hparams_per_fold[inner_fold["inner_fold"]]["gamma"], 
                                 alpha=hparams_per_fold[inner_fold["inner_fold"]]["alpha"])

        model.fit(
            inner_fold["train_ds"],
            epochs = EPOCHS_PER_FOLD,
            validation_data=inner_fold["val_ds"],
            callbacks=[tboard_callback],
            steps_per_epoch=8
        )
        
        vals = model.evaluate(inner_fold["test_ds"])
        inner_scores.append(vals[1])
    
    mean_test_acc = np.mean(inner_scores)
    print(f"Inner CV mean test acc: {mean_test_acc:.3f}")
    
    # Retrain with the best parameters
    best_inner = np.argmax(inner_scores)
    
    model = build_binary_cnn(input_shape=(128, 188, 1), 
                                lr=hparams_per_fold[best_inner]["lr"], 
                                gamma=hparams_per_fold[best_inner]["gamma"], 
                                alpha=hparams_per_fold[best_inner]["alpha"])
    tf.profiler.experimental.start('logs/')
    model.fit(
        outer_fold["inner"][best_inner]["train_ds"],
        epochs = EPOCHS_PER_FOLD,
        validation_data = outer_fold["inner"][best_inner]["val_ds"],
        callbacks=[tboard_callback]
    )
    tf.profiler.experimental.stop()
    vals = model.evaluate(outer_fold["test_ds"])
    per_fold_results.append(vals[1])

print(per_fold_results)

ERROR: Failed to launch TensorBoard (exited with 2).
Contents of stderr:
/home/joris/.pyenv/versions/3.12.11/lib/python3.12/site-packages/tensorboard/default.py:30: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
2025-08-29 11:19:10.158198: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
usage: tensorboard [-h] [--helpfull] [--logdir PATH] [--logdir_spec PATH_SPEC]
                   [--host ADDR] [--bind_all] [--port PORT]
                   [--reuse_port BOOL] [--load_fast {false,auto,true}]
                   [--extra_data_server

ERROR:root:Failed to start profiler: Another profiler is running.


[INFO]: Running outer fold: 1
[INFO]: Running inner fold: 1
Epoch 1/5


ERROR:root:Failed to start profiler: Another profiler is running.


8/8 ━━━━━━━━━━━━━━━━━━━━ 24s 1s/step - acc: 0.5913 - auc: 0.4627 - auprc: 0.3183 - loss: 0.2604 - precision: 0.3208 - recall: 0.2024 - val_acc: 0.3333 - val_auc: 0.5206 - val_auprc: 0.3897 - val_loss: 0.7958 - val_precision: 0.3333 - val_recall: 1.0000
Epoch 2/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 97ms/step - acc: 0.6627 - auc: 0.5477 - auprc: 0.4164 - loss: 0.2175 - precision: 0.4909 - recall: 0.3214 - val_acc: 0.6190 - val_auc: 0.4835 - val_auprc: 0.3587 - val_loss: 0.2841 - val_precision: 0.2727 - val_recall: 0.0857
Epoch 3/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - acc: 0.6786 - auc: 0.6195 - auprc: 0.4395 - loss: 0.2063 - precision: 0.5484 - recall: 0.2024 - val_acc: 0.3333 - val_auc: 0.5106 - val_auprc: 0.3690 - val_loss: 1.9299 - val_precision: 0.3333 - val_recall: 1.0000
Epoch 4/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 79ms/step - acc: 0.6865 - auc: 0.6519 - auprc: 0.5418 - loss: 0.1960 - precision: 0.5510 - recall: 0.3214 - val_acc: 0.3333 - val_auc: 0.5090 - val_auprc: 0.3346 - val_loss: 0.3634 

...

/home/joris/.pyenv/versions/3.12.11/lib/python3.12/site-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


8/8 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - acc: 0.5424 - auc: 0.5269 - auprc: 0.4030 - loss: 0.3000 - precision: 0.3226 - recall: 0.3390 - val_acc: 0.6667 - val_auc: 0.3360 - val_auprc: 0.2677 - val_loss: 0.4604 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 78ms/step - acc: 0.5537 - auc: 0.5045 - auprc: 0.3457 - loss: 0.1584 - precision: 0.3571 - recall: 0.4237 - val_acc: 0.3333 - val_auc: 0.6529 - val_auprc: 0.4539 - val_loss: 0.5453 - val_precision: 0.3333 - val_recall: 1.0000
Epoch 3/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step - acc: 0.6497 - auc: 0.5444 - auprc: 0.3856 - loss: 0.1482 - precision: 0.4545 - recall: 0.2542 - val_acc: 0.6667 - val_auc: 0.3356 - val_auprc: 0.2633 - val_loss: 0.6194 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - acc: 0.6497 - auc: 0.6337 - auprc: 0.4260 - loss: 0.1332 - precision: 0.4681 - recall: 0.3729 - val_acc: 0.6667 - val_auc: 0.3350 - val_auprc: 0.2669 - v

ERROR:root:Failed to start profiler: Another profiler is running.


8/8 ━━━━━━━━━━━━━━━━━━━━ 14s 580ms/step - acc: 0.5781 - auc: 0.5130 - auprc: 0.3373 - loss: 0.1244 - precision: 0.3556 - recall: 0.3902 - val_acc: 0.6667 - val_auc: 0.3507 - val_auprc: 0.2650 - val_loss: 0.0998 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 6s 43ms/step - acc: 0.4828 - auc: 0.5409 - auprc: 0.4463 - loss: 0.0938 - precision: 0.4000 - recall: 0.3077 - val_acc: 0.6667 - val_auc: 0.3607 - val_auprc: 0.3017 - val_loss: 0.0912 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step - acc: 0.6133 - auc: 0.5506 - auprc: 0.3667 - loss: 0.0928 - precision: 0.3924 - recall: 0.3780 - val_acc: 0.3333 - val_auc: 0.6628 - val_auprc: 0.5204 - val_loss: 0.1440 - val_precision: 0.3333 - val_recall: 1.0000
Epoch 4/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - acc: 0.5172 - auc: 0.5264 - auprc: 0.4644 - loss: 0.1035 - precision: 0.4286 - recall: 0.2308 - val_acc: 0.3333 - val_auc: 0.6758 - val_auprc: 0.5254 

AlreadyExistsError: Another profiler is running.